In [1]:
import pandas as pd

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

In [3]:
import xgboost as xgb

In [4]:
df_train = pd.read_csv("input/train_dataset.csv")

In [5]:
df_train.head(2)

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,...,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
0,6875768,4G,Smartphone,MALE,Kegalle,40-50,219,761.62,6317.0,2.773958,...,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,NaN,NaN,PKG2
1,6406277,4G,Smartphone,MALE,Galle,40-50,36,2482.52,28365.0,3.079438,...,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0,PKG1


In [6]:
df_train.shape

(10500, 31)

In [8]:
df_train['age_group'].value_counts()

30-40    3522
40-50    2585
20-30    1833
50-60    1412
60-70     701
>70       253
<20        59
Name: age_group, dtype: int64

In [9]:
map_dict_age_grp = {'30-40': '30_40', '40-50': '40_50', '20-30': '20_30', '50-60': '50_60', '60-70': '60_70',
                    '>70': 'more_than_70', '<20': 'less_than_20'}
df_train['age_group'].replace(map_dict_age_grp, inplace=True)

In [10]:
df_train['age_group'].value_counts()

30_40           3522
40_50           2585
20_30           1833
50_60           1412
60_70            701
more_than_70     253
less_than_20      59
Name: age_group, dtype: int64

In [7]:
df_plan = pd.read_csv("input/mobile_plans.csv")

In [148]:
df_plan.head(8)

,plan_name,data_quota,voice_onnet_quota,voice_offnet_quota,text_onnet_quota,text_offnet_quota,plan_value,plan_number
0,PKG1,3,900,0,900,0,400,1
1,PKG2,6,900,0,900,0,620,2
2,PKG3,8,unlimited,0,unlimited,0,950,3
3,PKG4,12,unlimited,0,unlimited,0,1300,4
4,PKG5,12,unlimited,450,unlimited,0,1550,5
5,PKG6,20,unlimited,450,unlimited,0,1900,6
6,PKG7,40,unlimited,450,unlimited,0,2550,7
7,PKG8,50,unlimited,900,unlimited,0,3200,8


In [149]:
len(df_plan)

8

In [150]:
df_plan['plan_number'].value_counts()

1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
Name: plan_number, dtype: int64

In [151]:
df_train.next_month_plan.value_counts()

PKG1    3673
PKG3    1717
PKG5    1558
PKG2    1354
PKG6     740
PKG4     735
PKG7     364
PKG8     359
Name: next_month_plan, dtype: int64

In [152]:
df_test = pd.read_csv("input/test_dataset.csv")

In [153]:
df_test.head(2)

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,...,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,2003793,4G,Smartphone,MALE,Colombo,40-50,4,689.42,148.0,0.000000,...,0.0,0.000000,0.0,0.0,NaN,0.000000,0.0,1.0,NaN,NaN
1,1776101,4G,Smartphone,MALE,Colombo,40-50,7,799.93,300.0,0.180222,...,107.0,16.266667,0.0,21.0,5.107143,4.924111,0.0,0.0,NaN,NaN


In [154]:
df_test.shape

(4500, 30)

In [155]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           10500 non-null  int64  
 1   device_type                  10477 non-null  object 
 2   device_category              10470 non-null  object 
 3   gender                       10365 non-null  object 
 4   district_name                10500 non-null  object 
 5   age_group                    10365 non-null  object 
 6   network_stay                 10500 non-null  int64  
 7   average_monthly_bill_amount  10500 non-null  float64
 8   dusage_sum                   10500 non-null  float64
 9   dusage_min                   10500 non-null  float64
 10  dusage_max                   10500 non-null  float64
 11  dusage_avg                   9642 non-null   float64
 12  dusage_days                  10500 non-null  float64
 13  dusage_stddev   

In [156]:
df_train["next_month_plan"].isna().sum()

0

In [157]:
df_train.isna().sum()

primary_identifier                0
device_type                      23
device_category                  30
gender                          135
district_name                     0
age_group                       135
network_stay                      0
average_monthly_bill_amount       0
dusage_sum                        0
dusage_min                        0
dusage_max                        0
dusage_avg                      858
dusage_days                       0
dusage_stddev                     0
vusage_onnet_sum                  0
vusage_onnet_max                  0
vusage_onnet_min                  0
vusage_onnet_days                 0
vusage_onnet_avg               8672
vusage_onnet_stddev               0
vusage_offnet_sum                 0
vusage_offnet_max                 0
vusage_offnet_min                 0
vusage_offnet_days                0
vusage_offnet_avg              3313
vusage_offnet_stddev              0
number_of_fixed_bb_accounts       0
number_of_iptv_accounts     

In [158]:
df_train["next_month_plan"].value_counts()

PKG1    3673
PKG3    1717
PKG5    1558
PKG2    1354
PKG6     740
PKG4     735
PKG7     364
PKG8     359
Name: next_month_plan, dtype: int64

In [159]:
# convert target values in numeric

map_dict_target = {'PKG1': 1, 'PKG2': 2, 'PKG3':3, 'PKG4': 4, 'PKG5': 5, 'PKG6': 6, 'PKG7': 7, 'PKG8': 8}
df_train['next_month_plan'].replace(map_dict_target, inplace=True)

In [160]:
df_train["next_month_plan"].value_counts()

1    3673
3    1717
5    1558
2    1354
6     740
4     735
7     364
8     359
Name: next_month_plan, dtype: int64

In [161]:
# select features
X = df_train.drop("next_month_plan", axis=1).copy()

In [162]:
X.shape

(10500, 30)

In [163]:
# target column
y = df_train['next_month_plan'].copy()

In [164]:
y.shape

(10500,)

In [165]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           10500 non-null  int64  
 1   device_type                  10477 non-null  object 
 2   device_category              10470 non-null  object 
 3   gender                       10365 non-null  object 
 4   district_name                10500 non-null  object 
 5   age_group                    10365 non-null  object 
 6   network_stay                 10500 non-null  int64  
 7   average_monthly_bill_amount  10500 non-null  float64
 8   dusage_sum                   10500 non-null  float64
 9   dusage_min                   10500 non-null  float64
 10  dusage_max                   10500 non-null  float64
 11  dusage_avg                   9642 non-null   float64
 12  dusage_days                  10500 non-null  float64
 13  dusage_stddev   

In [166]:
df_train["device_type"].value_counts()

4G    9380
2G     849
3G     248
Name: device_type, dtype: int64

In [167]:
df_train["device_category"].value_counts()

Smartphone        9532
Feature phone      542
Basic              318
Tablet              64
Modem                7
Pluggable card       7
Name: device_category, dtype: int64

In [168]:
df_train["gender"].value_counts()

MALE      7429
FEMALE    2936
Name: gender, dtype: int64

In [169]:
df_train["age_group"].value_counts()

30-40    3522
40-50    2585
20-30    1833
50-60    1412
60-70     701
>70       253
<20        59
Name: age_group, dtype: int64

In [170]:
df_train["district_name"].value_counts()

Colombo         4024
Gampaha         1946
Kandy            746
Kalutara         603
Kurunegala       436
Galle            377
Jaffna           280
Puttalam         259
Badulla          175
Matale           173
Kegalle          172
Ampara           166
Matara           149
Ratnapura        141
Anuradhapura     135
Batticaloa       130
Nuwara Eliya     109
Trincomalee      108
Hambantota        94
Polonnaruwa       64
Monaragala        62
Vavuniya          60
Kilinochchi       33
Mannar            24
Mullaitivu        19
Unknown           15
Name: district_name, dtype: int64

In [171]:
del df_train["age_group"]

In [172]:
X_encoded = pd.get_dummies(df_train, columns=[
    "device_type", "device_category", "gender",  "district_name"
])

In [173]:
X_encoded.shape

(10500, 63)

In [174]:
y.unique()

array([2, 1, 6, 4, 5, 3, 8, 7])

In [175]:
def train_test_split_stratified(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    return X_train, X_test, y_train, y_test

In [176]:
X_train, X_test, y_train, y_test = train_test_split_stratified(X_encoded, y)

In [177]:
X_train.shape, y_train.shape

((8400, 63), (8400,))

In [178]:
X_test.shape, y_test.shape

((2100, 63), (2100,))

In [179]:
# define xgb params

def xgboost_params():
    params = {
        'max_depth': 5,
        'eta': 0.3,
        'silent': 1,
        'objective': 'multi:softmax',
        'num_class': 8,
        'eval_metric': 'merror',
        'nthread': 4,
        'seed': 42,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'min_child_weight': 1,
        'gamma': 0.1,
        'reg_alpha': 0.1,
        'reg_lambda': 1,
        'scale_pos_weight': 1,
        'base_score': 0.5,
        'n_estimators': 100,
    }
    return params

In [180]:

# xgboost model for multi-class classification
def xgboost_model(X_train, X_test, y_train, y_test, params):
    # xgboost model
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # print(model.predict_proba(X_test))
    # print(model.predict_proba(X_test).shape)
    # print(y_pred)
    # print(y_pred.shape)
    # print(y_test)
    # print(y_test.shape)
    # print(model.score(X_test, y_test))
    return model, y_pred

In [181]:
params = xgboost_params()

In [182]:
model, y_pred = xgboost_model(X_train, X_test, y_train, y_test, params)

[03:46:30] WARNING: ../src/learner.cc:573: 
Parameters: { "scale_pos_weight", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/soumik/anaconda3/envs/lazuli/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [183]:
# calculate confusion matrix and f1 score for multi-class classification
def confusion_matrix_f1_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return cm, f1

# calculate macro f1 score for multi-class classification
def macro_f1_score(cm):
    f1 = 0
    for i in range(cm.shape[0]):
        f1 += cm[i][i] / np.sum(cm[i])
    f1 /= cm.shape[0]
    return f1

In [184]:
cm, f1  = confusion_matrix_f1_score(y_test, y_pred)

In [185]:
f1

1.0

In [186]:
macro_f1  = macro_f1_score(cm)

In [187]:
macro_f1

1.0

In [188]:
accuracy

1.0

In [189]:
y_pred

array([8, 1, 1, ..., 1, 1, 4])